#  爬网易云的评论
## 因为是POST方式，所以我们必须手动给出请求，但是网易云音乐参数是加密的，因此需要知道加密方法。
1.在payload中找到想要的未加密的参数                                     window.arsea(参数，xxxx,xxx,xx)
2.想办法把参数进行加密(必须参考网易的逻辑)，params，encSeckey            params -> encText,encSecKey -> encSecKey     p3 3分27秒
3.请求到网易，拿到评论信息

In [81]:
from Crypto.Cipher import AES  # AES解密
from base64 import b64encode   # 加密后的结果不能直接用utf-8转为字符串，因此要用b64转换
import requests
import json #将字典data变为字符串

In [82]:
url = "https://music.163.com/weapi/comment/resource/comments/get?csrf_token="

query = input("输入要搜索的结果")
data = {
    "csrf_token": "76de141d6a3132e9b86e0bfc00db6354",
    "cursor": "-1",
    "offset": "0",
    "orderType": "1",
    "pageNo": "1",
    "pageSize": "20",
    "rid": f"{query}",
    "threadId": f"{query}"
}
e = "010001"
f = "00e0b509f6259df8642dbc35662901477df22677ec152b5ff68ace615bb7b725152b3ab17a876aea8a5aa76d2e417629ec4ee341f56135fccf695280104e0312ecbda92557c93870114af6c9d05c4f7f0c3685b7a46bee255932575cce10b424d813cfe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7"
g = "0CoJUm6Qyw8W8jud"
i = "oZRMgqCksyJ7vLgF"   #通过程序随机运行一个随机数用来固定i

def get_encSecKey():
    return "a6b2cfa6491793414175006cff26a126f582d267e4238bc7beab2b9403e0c12ac5113c8920a50b404f457f982e2d219b965fb5fe61e27a4332080c4eb5cd2d4a6acc45152a614145734ebd727a1bd8d74de0c95c5946baf26843660a218e2e4af4a1bb446a9a3fd4c3fb850af124dc91bbec1185a64398adb397c86f333ebca3"
                # 由固定的i，一定会产生固定的encSecKey

def get_params(data):  #对应d函数求params的部分
    first = enc_params(data,g);
    second = enc_params(first,i);
    return second;

def to_16(data):
    pad = 16 - len(data)%16;
    data += chr(pad) * pad
    return data

def enc_params(data,key):  # 加密过程,对应b函数
    iv = "0102030405060708"
    data = to_16(data)
    aes = AES.new(key = key.encode("utf-8"),IV = iv.encode("utf-8"),mode = AES.MODE_CBC) #创建加密器
    bs = aes.encrypt(data.encode("utf-8")) #加密,加密的内容长度必须是16的倍数，
    return str(b64encode(bs),"utf-8") #对应 tostring

In [83]:
# 处理加密过程
#     function a(a) {  # 返回随机的16位字符串（a=16）
#         var d, e, b = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", c = "";
#         for (d = 0; a > d; d += 1)  # 循环16次
#             e = Math.random() * b.length, # 随机数
#             e = Math.floor(e), # 取整
#             c += b.charAt(e);  # 取字符串中的xxx位置
#         return c
#     }
#     function b(a, b) { # a是要加密的内容
#         var c = CryptoJS.enc.Utf8.parse(b) # b 是秘钥，根据b的方程可知g和i是秘钥
#           , d = CryptoJS.enc.Utf8.parse("0102030405060708")
#           , e = CryptoJS.enc.Utf8.parse(a)  # e是数据
#           , f = CryptoJS.AES.encrypt(e, c, {  # c是加密的秘钥，所以b是秘钥
#             iv: d,  # AES算法中的偏移量
#             mode: CryptoJS.mode.CBC # cbc加密模式
#         });
#         return f.toString()
#     }
#     function c(a, b, c) {
#         var d, e;
#         return setMaxDigits(131),
#         d = new RSAKeyPair(b,"",c),
#         e = encryptedString(d, a)
#     }
#     function d(d, e, f, g) { #d：数据(data) ,  e: 将buV2x(["流泪", "强"]代码在console运行后得到固定值010001 ,  f ： 同理得很长 , g :  0CoJUm6Qyw8W8jud
#         var h = {}
#           , i = a(16);  # i为16位的随机值
#         h.encText = b(d, g),    # g 和 i 是秘钥
#         h.encText = b(h.encText, i),  # 返回的就是params
#         h.encSecKey = c(i, e, f),     # 返回的就是encSecKey ，e 和 f是定死的, 如果将i设置为定值，那么c也是定值
#         return h
#     }
    

params进行了两次加密
数据加g -> b -> 第一次加密+i ->b = params
知道了加密算法后，我们就可以进行程序的编写

In [84]:
uers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36",
        "referer":"https://www.baidu.com/"}
proxies = {
    "https":"https://222.79.63.199:9999"
}
resp = requests.post(url,data={
    "params":get_params(json.dumps(data)),
    "encSecKey":get_encSecKey()
},headers=uers,timeout=1)
resp.encoding = 'utf-8'

# resp.encoding = resp.apparent_encoding
dic = resp.json()
              # with open("a.txt",mode="wb") as f:
              #     f.write(a.encode("utf-8"))
comments = dic['data']['hotComments']
for comment in comments:
    content = comment['content']
    print(content)

喜不喜欢，爱不爱，合不合适，在不在一起，住不住一块，有没有名分，过不过得下去，是七件事。
恋爱时也曾发誓不在婚姻里委曲求全，可真的嫁给对的那个人，才明白结婚不是两个人的事，面对他的家人，何止要委曲求全，忍辱负重也是有的。想在婚姻中幸福不需计较太多得失，因为关灯后背过身流下的泪水只有枕头才知道。
又是一个四不像的故事 需要用演才能维持的婚姻 真的能算是幸福吗[流泪]
麦浚龙监制！我想到罗生门女主其实过得并不好。哈哈哈哈哈哈
时光之中 见你便觉钟意 就是你了
如果我爱你令你压抑个性
抱歉我宁愿你从没遇上我
不想你个性为我改变什么
我爱你却是爱你的个性呀
这首歌的歌词很值得薛之谦学习，都是写“演员”，《人妻的伪术》太深入和丰富。
好久没有听甘正又好听的粤语歌，广州仔噶我支持你《谢安琪》♥
别为了不属于你的观众，去演绎你不擅长的人生。
婚姻里谁没有让步和掩饰，这歌唱的只是幸福的代价罢了。
我今天有点爱你    明天我说不清
我有强迫症，可以帮我点成双数吗[口罩]
相知 相识 相恋 相伴 然而我却觉得互相了解 互相了解的知音 比你侬我侬 陪伴依赖的恋人更难寻 大概世界最难得的事就是最好的朋友恰好也是我的恋人吧
做到第一件是缘分，第二件是真心，第三件是走心，第四件是想继续，第五件是愿意袒露一切，第六件是你值不值，而第七件则是靠命。事实上，我们能完成一半，就应该感谢自己好命了。
她宁愿收起个性，心甘情愿去做别人的贤内助，也不要和你相约冰岛，死心啦，麦浚龙
